In [ ]:
# THIS NOTEBOOK SHOWS EXAMPLES OF CUSTOM AND PYTHON FUNCTIONS APPLICATION FOR THE PURPOSES OF MRI PROJECT

In [ ]:
# Expanding visibility zone for Python engine to make HH Modules seen
import sys 
sys.path.append('../..')

In [ ]:
# Importing functions from hh_dates HH Module
from HH_Modules.hh_dates import hh_create_bus_dates
from HH_Modules.hh_dates import hh_drop_nyse_closures
from HH_Modules.hh_mri import hh_import_mri_source

In [ ]:
# Importing standard modules and date-special modules
import numpy as np
import pandas as pd
from datetime import date
from datetime import timedelta

In [ ]:
# Examples of using hh_dates function hh_create_bus_dates
date_format = '%Y-%m-%d'
null_date_ML_number = 693962
null_date_stamp = date(1900, 1, 1)
first_date_stamp = date(1998, 1, 1)
last_date_stamp = date(2018, 12, 31)
null_date = null_date_stamp.strftime(date_format)
first_date = first_date_stamp.strftime(date_format)
last_date = last_date_stamp.strftime(date_format)

date_index_days_str = hh_create_bus_dates('string', first_date, last_date, 'day')
date_index_weeks_str = hh_create_bus_dates('string', first_date, last_date, 'week')
date_index_monthes_str = hh_create_bus_dates('string', first_date, last_date, 'month')
date_index_quarters_str = hh_create_bus_dates('string', first_date, last_date, 'quarter')
date_index_years_str = hh_create_bus_dates('string', first_date, last_date, 'year')

date_index_days_date = hh_create_bus_dates('date', first_date_stamp, last_date_stamp, 'day')
date_index_weeks_date = hh_create_bus_dates('date', first_date_stamp, last_date_stamp, 'week')
date_index_monthes_date = hh_create_bus_dates('date', first_date_stamp, last_date_stamp, 'month')
date_index_quarters_date = hh_create_bus_dates('date', first_date_stamp, last_date_stamp, 'quarter')
date_index_years_date = hh_create_bus_dates('date', first_date_stamp, last_date_stamp, 'year')

print('First date: ', first_date)
print('Last date: ', last_date)
print('Business days (string type): ', date_index_days_str)
print('Business weeks ends (date type): ', date_index_weeks_date)
print('Business monthes ends (string type): ', date_index_monthes_str)
print('Business quarters ends (date type): ', date_index_quarters_date)
print('Business years ends (string type): ', date_index_years_str)

In [ ]:
# Generating Timeseries for comparing MatLab and Python hh_create_bus_dates functionality
calendar_delta = first_date_stamp - null_date_stamp

date_index_calendar_days = pd.date_range(first_date, last_date, freq = 'D')

ser_all_days = pd.Series(np.arange(date_index_calendar_days.size) + null_date_ML_number + calendar_delta.days, 
                         index = date_index_calendar_days.date, name = 'ML Numbers')

ser_bus_days = ser_all_days[date_index_days_str]
ser_bus_days.index.name = 'Business Dates'
ser_bus_days

In [ ]:
# Generating xlsx file for comparing MatLab and Python hh_create_bus_dates functionality on example of business days daily
xls_writer = pd.ExcelWriter('Data_Files/Test_Files/bus_dts.xlsx')
ser_bus_days.to_excel(xls_writer, 'daily')
xls_writer.save()

In [ ]:
# Examples of using hh_dates function hh_drop_nyse_closures
calendar_delta = first_date_stamp - null_date_stamp

date_nyse_index_days = hh_drop_nyse_closures(date_index_days_str)
ser_bus_nyse_days = pd.Series(0, index = date_nyse_index_days.date, name = 'ML Numbers')
ser_bus_nyse_days.index.name = 'Business Dates Wihtout NYSE Closures'
ser_bus_nyse_days

In [ ]:
# Generating xlsx file for comparing MatLab and Python hh_drop_nyse_closures functionality on example of business days daily
xls_writer = pd.ExcelWriter('Data_Files/Test_Files/bus_nyse_dts.xlsx')
ser_bus_nyse_days.to_excel(xls_writer, 'daily')
xls_writer.save()
# identified one discrepancy: 2018-12-05 - NYSE closure day for Python library and valid business day for MatLab function

In [ ]:
###################################################################################################################################

In [ ]:
df_model = hh_import_mri_source('Data_Files/Source_Files/mri_data.xlsx', 'Model 01', date_nyse_index_days)

In [14]:
df_model

,Asset Group,Asset Group Description,Asset Code,Asset Tab Name,Asset Desc,Processing Routine,Factor Weights
0,EQ,MRI Equity IVol Grp,iv_us,VIX,CBOE SPX VOLATILITY INDX,x,0.166667
1,EQ,MRI Equity IVol Grp,iv_eu,V2X,VSTOXX Index,x,0.166667
2,EQ,MRI Equity IVol Grp,iv_uk,VFTSE,FTSE100 Volatility Index,x,0.166667
3,EQ,MRI Equity IVol Grp,iv_jp,VNKY,NIKKEI Volatility Index,x,0.166667
4,EQ,MRI Equity IVol Grp,iv_rvx,RVX,CBOE RSL2000 VOLATILTY I,x,0.166667
5,EQ,MRI Equity IVol Grp,iv_eem,VXEEM,CBOE EM ETF Volatility,x,0.166667
7,FI,MRI Fixed Income Grp,oas_em,EMUSOAS,Bloomberg Barclays EM USD Aggr,x,0.500000
8,FI,MRI Fixed Income Grp,oas_hy,LF98OAS,Bloomberg Barclays US Corporat,x,0.500000
10,FX,MRI FOREX IVol Grp,fx_gbp,GBPUSD,GBP-USD OPT VOL 3M,x,0.250000
11,FX,MRI FOREX IVol Grp,fx_eur,EURUSD,EUR-USD OPT VOL 3M,x,0.250000


0         VIX
1         V2X
2       VFTSE
3        VNKY
4         RVX
5       VXEEM
7     EMUSOAS
8     LF98OAS
10     GBPUSD
11     EURUSD
12     USDJPY
13     USDCHF
Name: Asset Tab Name, dtype: object